# 简述

* 使用框架为eggjs和layui

### 一. 防止csrf攻击的安全策略

* 配置eggjs的config


* 进入项目文件，打开config/config.default.js,路径如图所示


* 编辑文件加入：
```javascript
config.security = {
    csrf: {
      enable: true,
    },
  };
// 声明中间件
config.middleware = [ 'errorHandler', 'authHandler' ];
```

* 编写中间件，例如:authHandler.js,在代码中声明

```javascript
ctx.state.csrf = ctx.csrf;
```

* form表单中加入
```javascript
<input type="hidden" name="_csrf" value="<%=csrf%>" />
```

* 其他的方式传输 csrftoken 方法
```javascript
// 获得指定名字存在cookie中的值
function getCookie(cookie_name) {
        var allcookies = document.cookie;
        var cookie_pos = allcookies.indexOf(cookie_name);
        if (cookie_pos != -1) {
            cookie_pos = cookie_pos + cookie_name.length + 1;
            var cookie_end = allcookies.indexOf(";", cookie_pos);
            if (cookie_end == -1) {
                cookie_end = allcookies.length;
            }
            var value = unescape(allcookies.substring(cookie_pos, cookie_end));
        }
        return value;
    }
// 第一种，$.ajax请求
$.ajax({
    type: "DELETE",
    url: "/deleteGrade",
    dataType: "json",
    data: {
        ids: obj.ids,
    },
    // 发送之前，加入请求头
    beforeSend: function (xhr) {
        xhr.setRequestHeader('x-csrf-token', csrftoken);
    },
    success: function (data, textStatus) {
        layer.close(loadIndex);
        console.log(data);
        if (data.code == 200) {
            layer.msg('删除成功');
            insTb.reload({ page: { curr: 1 } });
        } else {
            layer.msg(res.msg);
        }
    },
});
// 第二种，layui的upload方法
upload.render({
    elem: '#face-upload',
    accept: 'image',
    url: '/updateEmbedding',
    // 设置消息头
    headers: {'x-csrf-token': csrftoken},
    size: 1024,
    ...
});
```

### 二.等待异步执行完成

业务：接收图片跨域请求api进行更新数据

面对的问题：
1. 跨域请求需要request模块


2. 普通的request.post方法是异步的，所以没办法得到返回值，需要等待执行完成


3. 后端跨域问题

解决：
1. 安装插件
```code
npm install --save request
```


2. 使用Promise方法等待返回

``` javascript
async a() {
  return new Promise((resolve, reject) => function() {
    request.post({url: url, data; data}, function(error, res, body) {
      body = JSON.parse(body);
      resolve(body)
    }
  })
}
                     
async deal() {
  const body = await a(); 
}
    
// 还可以继续处理
async a() {
  return new Promise((resolve, reject) => function() {
    request.post({url: url, data; data}, function(error, res, body) {
      body = JSON.parse(body);
      resolve(body)
    }
  })
}.then(val = > {})
                     
// 或者
async deal() {
  let res = a(); 
  res.then(function(data) {});    
}
```

3. 后端跨域

第一步：安装插件:
```code
npm install egg-cors -g
```
第二步：在config/pluginjs中开启
```javascript
module.exports = {
	//其它插件...
	cors: {
	  enable: true,
	  package: 'egg-cors'
	}
	//其它插件...
}
```
第三步：在/config/config.default.js中对其进行配置
```javascript
config.cors = {
  origin: 'http://localhost:8080',//匹配规则  域名+端口  *则为全匹配
  allowMethods: 'GET,HEAD,PUT,POST,DELETE,PATCH'
};
```

### 三.使用vscode对eggjs进行Debug方法

1. 项目目录创建.vsocde文件夹，创建完后创建launch.json文件


2. 写入以下配置:
```code
{
    "version": "0.2.0",
    "configurations": [
      {
        "name": "Launch Egg",
        "type": "node",
        "request": "launch",
        "cwd": "${workspaceRoot}",
        "runtimeExecutable": "npm",
        "windows": { "runtimeExecutable": "npm.cmd" },
        "runtimeArgs": [ "run", "debug" ],
        "console": "integratedTerminal",
        "protocol": "auto",
        "restart": true,
        "port": 9999
      }
    ]
  }
```

3. 打断点按F5开始Debug